# 知识工程-作业7 中文语法错误纠正
2024214500 叶璨铭


## 代码与文档格式说明

> 本文档使用Jupyter Notebook编写，遵循Diátaxis 系统 Notebook实践 https://nbdev.fast.ai/tutorials/best_practices.html，所以同时包括了实验文档和实验代码。

> 本次实验项目采用了类似于 Quarto + nbdev 的方法来同步Jupyter Notebook代码到python文件, 因而我们的实验文档导出为pdf和html格式可以进行阅读，而我们的代码也导出为python模块形式，可以作为代码库被其他项目使用。
我们这样做的好处是，避免单独管理一堆 .py 文件，防止代码冗余和同步混乱，py文件和pdf文件都是从.ipynb文件导出的，可以保证实验文档和代码的一致性。

> 本文档理论上支持多个格式，包括ipynb, html, docx, pdf, md 等，但是由于 quarto和nbdev 系统的一些bug，我们目前暂时只支持ipynb, docx, pdf文件，以后有空的时候解决bug可以构建一个[在线文档网站](https://thu-coursework-machine-learning-for-big-data-docs.vercel.app/)。您在阅读本文档时，可以选择您喜欢的格式来进行阅读，建议您使用 Visual Studio Code (或者其他支持jupyter notebook的IDE, 但是VSCode阅读体验最佳) 打开 `ipynb`格式的文档来进行阅读。


> 为了记录我们自己修改了哪些地方，使用git进行版本控制，这样可以清晰地看出我们基于助教的代码在哪些位置进行了修改，有些修改是实现了要求的作业功能，而有些代码是对助教的代码进行了重构和优化。我将我在知识工程课程的代码，在作业截止DDL之后，开源到 https://github.com/2catycm/THU-Coursework-Knowledge-Engineering.git ，方便各位同学一起学习讨论。


## 代码规范说明

在我们实现函数过程中，函数的docstring应当遵循fastai规范而不是numpy规范，这样简洁清晰，不会Repeat yourself。相应的哲学和具体区别可以看 
https://nbdev.fast.ai/tutorials/best_practices.html#keep-docstrings-short-elaborate-in-separate-cells


为了让代码清晰规范，在作业开始前，使用 `ruff format`格式化助教老师给的代码; 

![alt text](image.png)


哇！我们当场就检查出了代码错误！不只是格式化问题了，看看 metrics/m2scorer/Tokenizer.py:177:15 和 metrics/m2scorer/token_offsets.py:43:15 是怎么回事

![alt text](image-2.png)

![alt text](image-1.png)


原来是 m2scorer 太老了，居然用了 Python2 的语法！我们简单修改为 `print("")` 语法就可以了。
不过语句有点多啊，我们一个个改有点不够优雅。

Python官方有工具，
```bash
2to3 -w .
```

![alt text](image-3.png)

改了特别多东西

![alt text](image-4.png)

终于勉强看起来正常了，不过细看代码还是有很多不正常的地方，看来学长只是想让我们参考代码，这个应该是难以跑通的。



同时注意到VSCode-Pylance插件的报错



## 实验环境准备


上次作业结束的时候，我们注意到我们想要尝试的最新方法只能支持3.12，PyTorch2.4也不够新，所以我们这次作业重新创建一个作业专属3.12环境。

先装小依赖包, 然后安装最新pytorch

```bash
conda create -n assignments python=3.12
conda activate assignments
pip install -r ../requirements.txt
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
```

注意到

```python
from elmoformanylangs import Embedder, logger
import einops
import jieba
```

参考 https://github.com/HIT-SCIR/ELMoForManyLangs， 安装

```bash
git submodule add https://github.com/HIT-SCIR/ELMoForManyLangs
cd ELMoForManyLangs
pip install -e .
```
PyTorch版本没有冲突，不用被乱装一通，太棒了。

安装一下上次没有探究完的 RWKV 
```bash
pip install -U git+https://github.com/TorchRWKV/flash-linear-attention
```


## 原理回顾和课件复习



课上详细介绍了语法纠正任务的一些基本特点和难点，先介绍了规则模型和分类模型，然后开始介绍翻译任务用于语法纠正和噪声信道模型，然后详细介绍了 Seq2seq 的发展历程，最后提及了一下大模型方法。


## 数据准备

download.sh 的清华网盘链接过期了，还好学长给的压缩包已经处理好了数据 NLPCC 2018 Task 2，已经有“./data/processed/”

```python
train_dataset = GECDataset(
    "./data/processed/seg.train", vocab_dict=vocab_dict, max_length=200
)
test_dataset = GECDataset(
    "./data/processed/seg.txt", vocab_dict=vocab_dict, max_length=200
)
```



## 预训练模型下载

根据main.py, 首先我们需要下载  "zhs.model"
```python
elmo_model = Embedder("zhs.model", batch_size=16)
vocab_dict = load_vocab_dict("./zhs.model/word.dic")
```

根据 https://github.com/HIT-SCIR/ELMoForManyLangs
有两个链接都是下载中文模型的，
```bash
wget http://vectors.nlpl.eu/repository/11/179.zip
wget http://39.96.43.154/zhs.model.tar.bz2
```
后者链接失效了无法连接。

## 评价指标 Max Match 实现

Max Match （因为有两个M 又叫作 M2 ）是由 NUS 的研究者在这篇2012 ACL 论文 https://aclanthology.org/N12-1067.pdf “Better Evaluation for Grammatical Error Correction” 提出的。

老师的课件指出这是应用最广泛的指标之一。

![alt text](image-5.png)


MaxMatch (M²) 首先第一个思路是不再直接比较生成句子和参考句子的相似度，而是比较系统所做的“编辑操作” (edits) 与人类标注者提供的“标准编辑操作”的匹配程度，也就是课件中说的ei和gi。

核心思想是“Max”，可以假设我们有多个标准答案，对于每一个人工标注的参考答案（即一个正确的句子版本），通过比较该参考答案和原始错误句子，找出从错误句子到这个特定正确版本所需的标准编辑操作。对于系统生成的句子所对应的一组编辑操作，M² 会尝试将其与每一个参考答案所对应的标准编辑操作集进行比较。

它会计算系统编辑集与每一个标准编辑集之间的匹配程度（通常是计算重叠的编辑数量）。
然后，它会选择那个能与系统编辑集产生最大匹配度（即最多重叠编辑）的参考答案。
重要的是， M² 认为，只要系统做出的编辑与 任意一个 正确的参考答案中的编辑相匹配，这个编辑就是有效的。这就是“MaxMatch”的含义——在所有可能的标准答案中，找到对系统最有利（匹配度最高）的那一个来进行评估。

M² 的主要目的是解决GEC评估中的模糊性问题：同一个错误修正问题可能有多种编辑操作序列导致相同的结果句子，而不同系统可能通过不同的编辑路径达到相同输出。


那么具体要怎么实现呢？经过我的初步调查，助教建议的第一个参考代码 https://github.com/shibing624/pycorrector 虽然看起来集成了很多方法，但是 max match 指标似乎没有（或者不叫这个名字），评测的代码是 “https://github.com/shibing624/pycorrector/blob/master/pycorrector/utils/evaluate_utils.py” 但是我还没有看懂，这里面引入了"SIGHAN", "句级评估结果", "设定需要纠错为正样本，无需纠错为负样本" 这些概念，好像用了对比学习，但是没有直接说是用 Max Match。

助教给我们的第二个代码就是 M² 的官方代码，也就是刚才我费了半天劲升级为 Python3 的代码。根据助教给的提示，我们可以直接外部调用其功能用来评估。


In [ ]:
from typing import *
import subprocess


def maxmatch_metric(prediction_file: str # a file containing predicted output
                    , label_file: str # a file containig groundtruth output
                    ) -> Any:
    """
    calculate maxmatch metrics

    File content example
    # prediction file
    ```
    冬 阴功 是 泰国 最 著名 的 菜 之一 ， 它 虽然 不 是 很 豪华 ， 但 它 的 味 确实 让 人 上瘾 ， 做法 也 不 难 、 不 复杂 。
    首先 ， 我们 得 准备 : 大 虾六 到 九 只 、 盐 一 茶匙 、 已 搾 好 的 柠檬汁 三 汤匙 、 泰国 柠檬 叶三叶 、 柠檬 香草 一 根 、 鱼酱 两 汤匙 、 辣椒 6 粒 ， 纯净 水 4量杯 、 香菜 半量杯 和 草菇 10 个 。
    ```
    # label_file
    ```
    S 冬 阴功 是 泰国 最 著名 的 菜 之一 ， 它 虽然 不 是 很 豪华 ， 但 它 的 味 确实 让 人 上瘾 ， 做法 也 不 难 、 不 复杂 。
    A 9 11|||W|||虽然 它|||REQUIRED|||-NONE-|||0

    S 首先 ， 我们 得 准备 : 大 虾六 到 九 只 、 盐 一 茶匙 、 已 搾 好 的 柠檬汁 三 汤匙 、 泰国 柠檬 叶三叶 、 柠檬 香草 一 根 、 鱼酱 两 汤匙 、 辣椒 6 粒 ， 纯净 水 4量杯 、 香菜 半量杯 和 草菇 10 个 。
    A 17 18|||S|||榨|||REQUIRED|||-NONE-|||0
    A 38 39|||S|||六|||REQUIRED|||-NONE-|||0
    A 43 44|||S|||四 量杯|||REQUIRED|||-NONE-|||0
    A 49 50|||S|||十|||REQUIRED|||-NONE-|||0
    ```
    """
    subprocess.check_call(
        ["python", "metrics/m2scorer/m2scorer.py", prediction_file, label_file]
    )


为此，我特意把助教注释里面的文件内容拿了出来进行测试，看看是否评测正确。

![alt text](image-6.png)

确实得是0， prediction_file 没有改错，这个是啥都不干的语法修正器。

然后我按照grond truth的要求，让LLM遵循ground truth的命令更正语法，得到 prediction_file2

![alt text](image-7.png)

没想到居然不是完全对，precision差一个，意思是我们不小心多改了一个什么东西。

仔细检查了半天，发现原来是助教给我们的Python 注释有问题，冬阴功分词给合起来了，我们修改一下 Python 注释以及 predicition_file_correct.txt 就行。

![alt text](image-8.png)

我们还可以写正则表达式提取一下这三个数值出来，这样返回的时候更好处理

![alt text](image-9.png)

In [ ]:
def maxmatch_metric(prediction_file: str # a file containing predicted output
                    , label_file: str # a file containig groundtruth output
                    , verbose:bool = True
                    ) -> Any:
    """
    calculate maxmatch metrics

    File content example
    # prediction file
    ```
    冬 阴功 是 泰国 最 著名 的 菜 之一 ， 它 虽然 不 是 很 豪华 ， 但 它 的 味 确实 让 人 上瘾 ， 做法 也 不 难 、 不 复杂 。
    首先 ， 我们 得 准备 : 大 虾六 到 九 只 、 盐 一 茶匙 、 已 搾 好 的 柠檬汁 三 汤匙 、 泰国 柠檬 叶三叶 、 柠檬 香草 一 根 、 鱼酱 两 汤匙 、 辣椒 6 粒 ， 纯净 水 4量杯 、 香菜 半量杯 和 草菇 10 个 。
    ```
    # label_file
    ```
    S 冬 阴功 是 泰国 最 著名 的 菜 之一 ， 它 虽然 不 是 很 豪华 ， 但 它 的 味 确实 让 人 上瘾 ， 做法 也 不 难 、 不 复杂 。
    A 9 11|||W|||虽然 它|||REQUIRED|||-NONE-|||0

    S 首先 ， 我们 得 准备 : 大 虾六 到 九 只 、 盐 一 茶匙 、 已 搾 好 的 柠檬汁 三 汤匙 、 泰国 柠檬 叶三叶 、 柠檬 香草 一 根 、 鱼酱 两 汤匙 、 辣椒 6 粒 ， 纯净 水 4量杯 、 香菜 半量杯 和 草菇 10 个 。
    A 17 18|||S|||榨|||REQUIRED|||-NONE-|||0
    A 38 39|||S|||六|||REQUIRED|||-NONE-|||0
    A 43 44|||S|||四 量杯|||REQUIRED|||-NONE-|||0
    A 49 50|||S|||十|||REQUIRED|||-NONE-|||0
    ```
    """
    # subprocess.check_call(
    #     ["python", "metrics/m2scorer/m2scorer.py", prediction_file, label_file]
    # )
    # 执行命令并捕获输出
    result = subprocess.check_output(
        ["python", "metrics/m2scorer/m2scorer.py", prediction_file, label_file],
        text=True  # 直接获取文本输出，无需解码
    )

    # 打印原始输出
    if verbose:
        print("m2scorer评测中:")
        print(result)

    # 使用正则表达式提取指标
    metrics = {}
    metrics_pattern = re.compile(r'(Precision|Recall|F0\.5)\s*:\s*([\d\.]+)')
    matches = metrics_pattern.findall(result)
    
    # 将匹配的结果转换为字典
    for key, value in matches:
        metrics[key] = float(value)

    return metrics

当然这样会被底层代码限制住，只能获得4位小数，虽然对我们这次实验够用了，但是为了避免后人再被这个m2scorer坑住，我们决定把里面的代码改造一些出来。

In [ ]:
from pathlib import Path
this_file = Path(__file__).resolve()
this_directory = this_file.parent
import sys
sys.path.append((this_directory/"m2scorer").as_posix())

import levenshtein as levenshtein
from util import paragraphs
from util import smart_open
from typing import Any

def load_annotation(gold_file):
    source_sentences = []
    gold_edits = []
    fgold = smart_open(gold_file, "r")
    puffer = fgold.read()
    fgold.close()
    # puffer = puffer.decode('utf8')
    for item in paragraphs(puffer.splitlines(True)):
        item = item.splitlines(False)
        sentence = [line[2:].strip() for line in item if line.startswith("S ")]
        assert sentence != []
        annotations = {}
        for line in item[1:]:
            if line.startswith("I ") or line.startswith("S "):
                continue
            assert line.startswith("A ")
            line = line[2:]
            fields = line.split("|||")
            start_offset = int(fields[0].split()[0])
            end_offset = int(fields[0].split()[1])
            etype = fields[1]
            if etype == "noop":
                start_offset = -1
                end_offset = -1
            corrections = [
                c.strip() if c != "-NONE-" else "" for c in fields[2].split("||")
            ]
            # NOTE: start and end are *token* offsets
            original = " ".join(" ".join(sentence).split()[start_offset:end_offset])
            annotator = int(fields[5])
            if annotator not in list(annotations.keys()):
                annotations[annotator] = []
            annotations[annotator].append(
                (start_offset, end_offset, original, corrections)
            )
        tok_offset = 0
        for this_sentence in sentence:
            tok_offset += len(this_sentence.split())
            source_sentences.append(this_sentence)
            this_edits = {}
            for annotator, annotation in annotations.items():
                this_edits[annotator] = [
                    edit
                    for edit in annotation
                    if edit[0] <= tok_offset
                    and edit[1] <= tok_offset
                    and edit[0] >= 0
                    and edit[1] >= 0
                ]
            if len(this_edits) == 0:
                this_edits[0] = []
            gold_edits.append(this_edits)
    return (source_sentences, gold_edits)


def maxmatch_metric(prediction_file: str # a file containing predicted output
                    , label_file: str # a file containig groundtruth output
                    , verbose:bool = True
                    ) -> Any:
    """
    calculate maxmatch metrics

    File content example
    # prediction file
    ```
    冬 阴功 是 泰国 最 著名 的 菜 之一 ， 它 虽然 不 是 很 豪华 ， 但 它 的 味 确实 让 人 上瘾 ， 做法 也 不 难 、 不 复杂 。
    首先 ， 我们 得 准备 : 大 虾六 到 九 只 、 盐 一 茶匙 、 已 搾 好 的 柠檬汁 三 汤匙 、 泰国 柠檬 叶三叶 、 柠檬 香草 一 根 、 鱼酱 两 汤匙 、 辣椒 6 粒 ， 纯净 水 4量杯 、 香菜 半量杯 和 草菇 10 个 。
    ```
    # label_file
    ```
    S 冬 阴功 是 泰国 最 著名 的 菜 之一 ， 它 虽然 不 是 很 豪华 ， 但 它 的 味 确实 让 人 上瘾 ， 做法 也 不 难 、 不 复杂 。
    A 9 11|||W|||虽然 它|||REQUIRED|||-NONE-|||0

    S 首先 ， 我们 得 准备 : 大 虾六 到 九 只 、 盐 一 茶匙 、 已 搾 好 的 柠檬汁 三 汤匙 、 泰国 柠檬 叶三叶 、 柠檬 香草 一 根 、 鱼酱 两 汤匙 、 辣椒 6 粒 ， 纯净 水 4量杯 、 香菜 半量杯 和 草菇 10 个 。
    A 17 18|||S|||榨|||REQUIRED|||-NONE-|||0
    A 38 39|||S|||六|||REQUIRED|||-NONE-|||0
    A 43 44|||S|||四 量杯|||REQUIRED|||-NONE-|||0
    A 49 50|||S|||十|||REQUIRED|||-NONE-|||0
    ```
    """
    max_unchanged_words = 2
    beta = 0.5
    ignore_whitespace_casing = False
    very_verbose = False

    # load source sentences and gold edits
    source_sentences, gold_edits = load_annotation(label_file)

    # load system hypotheses
    fin = smart_open(prediction_file, "r")
    system_sentences = [line.strip() for line in fin.readlines()]
    fin.close()

    p, r, f1 = levenshtein.batch_multi_pre_rec_f1(
        system_sentences,
        source_sentences,
        gold_edits,
        max_unchanged_words,
        beta,
        ignore_whitespace_casing,
        verbose,
        very_verbose,
    )

    metrics = {
        "Precision": p,
        "Recall": r,
        "F_{}".format(beta): f1
    }

    return metrics

# 如果需要测试函数，可以调用它并打印结果
if __name__ == "__main__":
    prediction_file = (this_directory/"../data/test_prediction_file_correct.txt").as_posix()
    label_file = (this_directory/"../data/test_label_file.txt").as_posix()
    metrics = maxmatch_metric(prediction_file, label_file)
    print("Metrics:", metrics)


![alt text](image-10.png)

看来实现是正确的，而且我们还能看到详细的信息。

## 数据加载



我们看下 main.py 文件

使用到数据的地方是

```python
from data_util import MyDataset, collate_fn
...
MyDataset(
        "./data/train.tsv",
        max_length=max_length,
        train=True,
        max_example_num=max_train_example,
    ),
collate_fn=functools.partial(collate_fn, device=device),
```

助教已经帮我们实现了 collate_fn 关键是我们要写 MyDataset 

首先注意到 MyDataset[i] 返回的是一个元组，包含文本和标签

```python
def __getitem__(self, item):
    return self.text[item], self.label[item]
```

需要自己有list来存。现在可以写 load 

我们需要观察一下数据格式

![alt text](image-2.png)

train.tsv 和 dev.tsv 都是  “sentence label”，test.tsv 是 “index sentence”， 这次作业 main.py 只要求我们做 train 和 dev的,c传递的参数都是 train=True。

注意 csv 是 "comma separated values" 的缩写，tsv 是 "tab separated values" 的缩写，csv文件的分隔符是逗号，而tsv文件的分隔符是制表符（tab），所以我们需要使用 `sep="\t"` 来读取数据。


In [ ]:
from typing import List, Tuple
import torch


def load(
    self,
    file: str,  # file path
    train: bool = True,  # whether is training file
) -> Tuple[List[List[str]], List[int]]:  # Returns (text, label), text input and label
    """
    load file into texts and labels
    """
    import pandas as pd

    # 使用pandas读取文件，自动推断分隔符
    try:
        # 首先尝试tab分隔符，因为这是期望的格式
        df = pd.read_csv(file, sep="\t")
    except Exception as e:
        print(f"Error reading file with tab separator: {e}")
        print("Trying to read with auto-detected separator...")
        # 如果失败，让pandas尝试自动推断分隔符
        df = pd.read_csv(file, sep=None, engine="python")

    text = df["sentence"].astype(str).tolist()
    # 分词
    text = [sentence.split() for sentence in text]

    if train:
        # 训练集格式: sentence  label
        label = df["label"].astype(int).tolist()
    else:
        # 测试集可能没有标签，默认为 -1 表示不知道
        label = [-1] * len(text)

    return text, label

注意助教对  Returns (text, label), text input and label 的类型标注有误。

首先label应该是 List[int] 类型，参考官方文档 https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html ，可以是float，但是不应该是str类型。

其次text不应该是 List[str], 这里我们需要查看allennlp的 batch_to_ids 函数的约定 https://github.com/allenai/allennlp/blob/main/allennlp/modules/elmo.py ，可以看到其要求的输入是 List[List[str]], 也就是说需要对句子进行分词！


![](image-3.png)

我们差点就被原本的注释带偏啦，还好检查了allennlp的文档。


分词并不难，因为这次作业是 “英文评论情感分类”，可以直接按照空白符分词，使用 `str.split()` 就可以了，不需要上次那样用结巴分词。

当然，如果做得细致些，应该用 elmo 的 tokenier 去做分词，或者用nltk。 


现在可以实现 pad 函数

事实上，刚才我们看了源码知道，allennlp的elmo已经实现了padding，会根据句子和单词的最大长度来补齐，实际上我们不应该做任何操作！

当然，助教给了我们一个 max_length 的参数，实际上是用来限制句子长度的，超过这个长度的句子会被截断, 如果比elmo从数据发现的最大长度还要长，那多补一些也无妨，我们还是能实现一个。

不过最重要的问题是，pad token是什么？上一次作业是助教定义的词库，传入了pad和unknown的id，这次作业我们需要遵循allennlp的elmo的约定！

这下我们不得不继续查看源码 https://github.com/allenai/allennlp/blob/main/allennlp/modules/elmo.py

![](image-4.png)

这下可以确认pad token是0，从而实现

In [ ]:
def pad(
    self,
    text_ids: torch.Tensor,  # size N*L*D
) -> torch.Tensor:  # Returns padded_text_id, size N*max_length*D
    """
    pad text_ids to max_length
    """
    N, L, D = text_ids.shape
    if L >= self.max_length:
        # 如果文本长度大于等于最大长度，截断
        return text_ids[:, : self.max_length, :]
    else:
        # 如果文本长度小于最大长度，填充
        padding = torch.zeros(
            N, self.max_length - L, D, dtype=text_ids.dtype, device=text_ids.device
        )
        padded_text_ids = torch.cat([text_ids, padding], dim=1)
        return padded_text_ids

测试一下, 用 https://github.com/google-deepmind/treescope 看清楚数据是什么样的

In [ ]:
import treescope

treescope.basic_interactive_setup(autovisualize_arrays=True)

In [ ]:
dataset = MyDataset("./data/dev.tsv")
t, l = dataset[0]

## CNN 神经网络实现

这次的TextCNN和第四次作业的基本一样，只是embedding换成了allennlp的elmo representation，然后分类数量换成了2，其他的都一样。

上次我写得代码已经比较优雅高效，更多关于这段代码的理解和实现逻辑参阅 上次作业报告 https://github.com/2catycm/THU-Coursework-Knowledge-Engineering/blob/master/4.%E6%96%B0%E9%97%BB%E6%96%87%E6%9C%AC%E5%88%86%E7%B1%BB%E7%9A%84%E4%BD%9C%E4%B8%9A/P_homework4.ipynb 

这次我们增加一个dropout

In [ ]:
class TextCNN(nn.Module):
    def __init__(
        self,
        options_file: str,  # elmo file
        weight_file: str,  # elmo weight file
        vector_size: int,  # word embedding dim
        filter_size: List[int] = [2, 3, 4, 5],  # kernel size for each layer of CNN
        channels: int = 64,  # output channel for CNN
        max_length: int = 1024,  # max length of input sentence
        dropout=0.5,  # dropout rate
    ):
        super(TextCNN, self).__init__()
        self.embedding = Elmo(options_file, weight_file, 1, dropout=0)
        ####################
        # 初始化嵌入层已经通过Elmo完成
        # 直接用上次作业的代码
        # Build a stack of 1D CNN layers for each filter size
        self.convs = nn.ModuleList(
            [
                nn.Conv1d(in_channels=vector_size, out_channels=channels, kernel_size=k)
                # Conv1dViaConv2d(
                #     in_channels=vector_size,
                #     out_channels=channels,
                #     kernel_size=k,
                #     conv_2d=KAN_Convolutional_Layer,
                # )
                for k in filter_size
            ]
        )
        # Final linear layer for label prediction; number of classes equals len(label2index)
        # CNN的输出通道数 × 不同卷积核的数量
        self.linear = nn.Linear(
            channels * len(filter_size), 2
        )  # 二分类问题（正面/负面）
        #  Dropout层，防止过拟合
        self.dropout = nn.Dropout(dropout)
        self.max_length = max_length

    def forward(
        self,
        inputs: torch.Tensor,  # input sentence, size N*L
    ) -> torch.Tensor:  # predicted_logits: torch.tensor of size N*C (number of classes)
        # 获取ELMo嵌入表示
        inputs = self.embedding(inputs)["elmo_representations"][
            0
        ]  # [N, L, vector_size]

        # Convolutional layer
        x = inputs.transpose(1, 2)  # 卷积需要将词向量维度放在最后 (N*D*L)
        x = [conv(x) for conv in self.convs]
        x = [nn.functional.gelu(i) for i in x]  # 每一个 i是 (N*C*Li) ， Li = L - ki + 1
        # Pooling layer
        x = [
            nn.functional.max_pool1d(
                i,
                kernel_size=i.size(2),  # 对 Li 去做 max_pooling
            ).squeeze(2)
            for i in x  # 每一个 i是 (N*C*Li)
        ]  # 每一个 item 变为 (N*C)
        # Concatenate all pooling results
        x = torch.cat(x, dim=1)  # 把每一个 item 拼接起来，变为 (N, C*len(filter_size))
        # 应用dropout
        x = self.dropout(x)
        # Linear layer
        x = self.linear(x)  # 分类，得到 (N*K)
        return x

其中我们还用了 Conv2d兼容层，来尝试实现 KAN Conv，但是并没有成功，这次老师讲解的重点是RNN，所以今天我们不过多探索Kolmogorov–Arnold Networks。

## RNN 神经网络实现

### LSTM 实现

PyTorch 官方已经实现LSTM块，直接调用就是最佳实践，优雅高效。 https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html

我们只需要确保接口和TextCNN一致就可以了，为了公平比较，embedding都是用elmo一样的设置。


In [ ]:
import torch
from torch import nn
from allennlp.modules.elmo import Elmo
from typing import List


class TextLSTM(nn.Module):
    def __init__(
        self,
        options_file: str,  # elmo options file
        weight_file: str,  # elmo weight file
        vector_size: int,  # word embedding dim
        filter_size: List[int] = [2, 3, 4, 5],  # 保留接口，与CNN一致，但不使用
        channels: int = 64,  # 作为LSTM隐藏层维度
        max_length: int = 1024,  # 最大句子长度
        dropout: float = 0.5,  # dropout rate
    ):
        super(TextLSTM, self).__init__()
        self.embedding = Elmo(options_file, weight_file, 1, dropout=0)
        # 使用LSTM进行特征抽取，使用channels作为隐藏层维度
        self.lstm = nn.LSTM(
            input_size=vector_size, hidden_size=channels, batch_first=True
        )
        self.dropout = nn.Dropout(dropout)
        # 最后全连接分类层
        self.linear = nn.Linear(channels, 2)  # 二分类问题

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        """
        利用ELMo嵌入和LSTM进行前向传播
        """
        # 获取ELMo嵌入表示, 输出形状为 (N, L, vector_size)
        x = self.embedding(inputs)["elmo_representations"][0]
        # 通过LSTM，输出h_n形状为 (num_layers, N, hidden_size)
        _, (h_n, _) = self.lstm(x)
        # 提取最后一层隐藏状态，形状为 (N, hidden_size)
        h = h_n[-1]
        # 应用Dropout
        h = self.dropout(h)
        # 分类
        out = self.linear(h)
        return out

其中对于LSTM的参数，batch_first 是因为我们elmo得到的顺序是 (N, L, vector_size)，N在前面。

input_size 和 hidden_size 有所不同，类似于CNN中的in_channels 和 out_channels。 



在forward中用的时候，输入 input, (h_0, c_0)
输出 output, (h_n, c_n)

h_0 和 c_0 可以不提供 ，“Defaults to zeros”



### RWKV 实现

RWKV 是 目前较为先进的RNN模型，号称结合了 Attention 并行训练和长文本建模的能力和RNN高效推理无限上下文长度的能力。RWKV 本质上和LSTM一样都是RNN模型，只是其使用了不同的激活函数和门控机制以及其他的一些高级操作。

感觉RWKV的开源做的很好，在linux基金会下 https://rwkv.cn/news/read?id=15 ，看起来很有前景。

RWKV-LM 的开源代码很复杂，有cuda kernel c++啥的，很难改。通过和RWKV社区成员沟通交流，发现这个https://github.com/TorchRWKV/flash-linear-attention/tree/stable 实现比较优雅，用torch写，但是用triton编译。

我们安装一下。

```bash
pip install -U git+https://github.com/TorchRWKV/flash-linear-attention
```

看源码 https://github.com/TorchRWKV/flash-linear-attention/blob/stable/fla/layers/rwkv7.py



In [ ]:
import torch
from torch import nn
from allennlp.modules.elmo import Elmo
from typing import List

# 引入 RWKV7Attention 模块
from fla.layers.rwkv7 import RWKV7Attention


class TextRWKV(nn.Module):
    def __init__(
        self,
        options_file: str,  # elmo选项文件
        weight_file: str,  # elmo权重文件
        vector_size: int,  # 词向量维度
        filter_size: List[int] = [2, 3, 4, 5],  # 保持接口一致，但不使用
        channels: int = 64,  # 保持接口一致，可用于其他用途
        max_length: int = 1024,  # 最大句子长度
        dropout: float = 0.5,  # dropout 概率
    ):
        super(TextRWKV, self).__init__()
        # 使用ELMo构造嵌入层
        self.embedding = Elmo(options_file, weight_file, 1, dropout=0)
        # 构造 RWKV 模块，采用 chunk 模式，hidden_size 使用 vector_size
        self.rwkv = RWKV7Attention(mode="chunk", hidden_size=vector_size, head_dim=64)
        self.dropout = nn.Dropout(dropout)
        # 最后全连接分类层，将特征映射到二分类问题
        self.linear = nn.Linear(vector_size, 2)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        """
        利用ELMo嵌入和RWKV7Attention模块进行前向传播

        Parameters:
            inputs: torch.Tensor
                输入句子（形状为 N x L）

        Returns:
            torch.Tensor: 分类预测 logits（形状为 N x 2）
        """
        # 获取ELMo嵌入表示，形状为 (N, L, vector_size)
        x = self.embedding(inputs)["elmo_representations"][0]
        # 通过RWKV模块，输出形状假设为 (N, L, vector_size)
        o, _, _, _ = self.rwkv(x)
        # 对时间步进行最大池化，得到 (N, vector_size) 表示
        h, _ = torch.max(o, dim=1)
        # 应用ReLU激活增强非线性，然后dropout
        h = torch.relu(h)
        h = self.dropout(h)
        # 全连接分类层，输出二分类 logits
        out = self.linear(h)
        return out

## 运行效果

首先我们直接运行一下CNN，为了让速度快一些，我改了batch size为16\*64，learning rate也\*16。

```bash
CUDA_VISIBLE_DEVICES=1 python main.py
```

![](image-5.png)

可以看到效果一般，只到80。

我们把batch size换回 64 再跑一次


在跑之前，我们修改一下 main.py 代码，第一，老规矩啦，要支持argparse；第二这一次助教用了tqdm，但是中间完全没有反馈，根本看不到网络训练地好不好，不知道刚才80的问题在哪，所以我们加上loss和acc的反馈

```python
out_bar = tqdm(range(total_epoch))
for epoch in out_bar: 
    ...
    bar = tqdm(train_loader)
    for text, label in bar:
        ...
        bar.set_description(f"Loss: {loss.item():.4f}")
    ...
    out_bar.set_description(f"Epoch: {epoch} Max Accuracy: {max_acc:.4f}")
```

好现在跑

![](image-6.png)

可以看到达到了 85.89，四舍五入勉强复现了助教说的 86， 那么问题有可能是dropout=0.5 太大。

现在关闭dropout，看看效果

```bash
CUDA_VISIBLE_DEVICES=1 python main.py --model cnn --batch_size 64 --dropout 0
```

![alt text](image-7.png)

得到85.21
看来还是有0.5的dropout性能更好。

### LSTM 的性能

```bash
CUDA_VISIBLE_DEVICES=6 python main.py --model lstm --batch_size 64 --dropout 0.5
```

![alt text](image-8.png)

效果大幅弱于 CNN！

关闭dropout后性能更弱了

![alt text](image-9.png)


仔细检查刚才上面的代码，我发现，相比于CNN，我们没有增加RELU函数。虽然根据课件，RNN自己有激活函数，下一个状态的hidden state是从上一个状态得到的，有激活函数。

但是为了TextCNN差不多，我们还是加上RELU函数吧，看看效果

同时把LSTM变成双向LSTM


In [ ]:
class TextLSTM(nn.Module):
    def __init__(
        self,
        options_file: str,  # elmo options file
        weight_file: str,  # elmo weight file
        vector_size: int,  # word embedding dim
        filter_size: List[int] = [2, 3, 4, 5],  # 保留接口，与CNN一致，但不使用
        channels: int = 64,  # 作为LSTM隐藏层维度
        max_length: int = 1024,  # 最大句子长度
        dropout: float = 0.5,  # dropout rate
    ):
        super(TextLSTM, self).__init__()
        self.embedding = Elmo(options_file, weight_file, 1, dropout=0)
        # 使用LSTM进行特征抽取，使用channels作为隐藏层维度
        self.lstm = nn.LSTM(
            input_size=vector_size,
            hidden_size=channels,
            batch_first=True,
            bidirectional=True,
        )
        self.dropout = nn.Dropout(dropout)
        # 最后全连接分类层
        self.linear = nn.Linear(2 * channels, 2)  # 二分类问题，双向LSTM输出

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        """
        利用ELMo嵌入和LSTM进行前向传播
        """
        # 获取ELMo嵌入表示, 输出形状为 (N, L, vector_size)
        x = self.embedding(inputs)["elmo_representations"][0]
        # 通过LSTM，输出h_n形状为 (num_layers, N, hidden_size)
        _, (h_n, _) = self.lstm(x)
        # 双向LSTM，h_n形状为 (num_directions, N, hidden_size)，拼接正反向的最后隐藏状态
        h = torch.cat([h_n[0], h_n[1]], dim=1)
        # 添加ReLU激活，增强非线性能力
        h = torch.relu(h)
        # 应用Dropout
        h = self.dropout(h)
        # 分类
        out = self.linear(h)
        return out

![alt text](image-10.png)

这下效果终于正常了。

在老师的课件中
![alt text](image-11.png)

有两种结构，我们用的是第二种，而且只有单层。

![alt text](image-12.png)

老师还提到CNN优于RNN也是正常的。

### RWKV 的性能


CUDA_VISIBLE_DEVICES=1 PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True python main.py --model rwkv

遇到报错
```bash
ImportError: cannot import name 'DeviceMesh' from 'torch.distributed.tensor' 
```

https://github.com/allenai/OLMo/issues/559

但是我们按照issue把PyTorch升级到最新，也还是报这个错。

看来RWKV不是那么容易跑啊。